# Publication Plots for the Partition Problem

In [ ]:
using HDF5, Printf
using LsqFit, Measurements
using Statistics, Distributions

using PyCall, PyPlot
np = pyimport("numpy")
h5py = pyimport("h5py")
PyPlot.plt.style.use("paper.mplstyle")

In [ ]:
# path to folder with the h5 files
PATH = "../data/number-partitioning/"

## Plots

### Average Energy

In [ ]:
p = 10000
num_instances = 10000
seed = 137;

In [ ]:
X = 10:2:20;
X = vcat(X, [25 + 5k for k in 0:4], [50, 60, 70, 80, 90]);

In [ ]:
E_star_data = []
for N in X
    new_file = "number-partitioning_" * "p_" * string(p) * "_N_" * string(N) * "_num_inst_" * string(num_instances) * "_seed_" * string(seed) * "_moments.h5"
    data_file = h5open(PATH * new_file, "r")
    println(new_file)
    E_star_sqrt = read(data_file, "E_star_sqrt")[1]
    push!(E_star_data, E_star_sqrt)
end

In [ ]:
m(x, p) = p[1] .+ p[2] .* x
p0 = [0., -0.5];

In [ ]:
fit_E = curve_fit(m, log.(X), log.(E_star_data), p0)    
println(fit_E.resid .|> abs |> sum)    
println(fit_E.param)

In [ ]:
exp(fit_E.param[1])

In [ ]:
fig = figure(figsize=(4.2, 2))
ax = subplot(111)
ax.plot(X, E_star_data, "o")
ax.plot(X, map(x -> m(x, [fit_E.param[1], fit_E.param[2]]), log.(X)) .|> exp, "-C0", label="\$AN^{-\\omega}\$")
ax.plot([], [], lw=0, label=@sprintf("\$A=%.1f,\\, \\omega=%.1f \$", fit_E.param[1] |> exp, fit_E.param[2] |> abs))
ax.set_xlabel("\$N\$")
ax.set_ylabel("\$\\langle\\sqrt{E}\\rangle\$")
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim(nothing, 1e2)
ax.set_ylim(1e-4, 1e-1)
ax.legend(loc="lower left", ncol=2, columnspacing=-0.5, handlelength=1)

tight_layout(pad=0.15, w_pad=0.0, h_pad=0.0)

### Histograms

In [ ]:
@pyimport matplotlib.gridspec as gspec

colors = ["C0", "k", "C2", "C3"];
alphas = [0.5, 0.3]

fig = plt.figure(figsize=(4.2, 5), constrained_layout=true)

spec = gspec.GridSpec(figure=fig, ncols=1, nrows=2, wspace=0.0, hspace=0.0, height_ratios=[1, 2])

ax1 = fig.add_subplot(spec[1])
ax2 = fig.add_subplot(spec[0])

ax1.plot(X, E_star_data, "o")
ax1.plot(X, map(x -> m(x, [fit_E.param[1], fit_E.param[2]]), log.(X)) .|> exp, "-C0", label="\$AN^{-\\omega}\$")
ax1.plot([], [], lw=0, label=@sprintf("\$A=%.1f,\\, \\omega=%.1f \$", fit_E.param[1] |> exp, fit_E.param[2] |> abs))
ax1.set_xlabel("\$N\$", labelpad=-4)
ax1.set_ylabel("\$\\langle\\sqrt{E_*}\\rangle\$")
ax1.set_xscale("log")
ax1.set_yscale("log")
ax1.set_xlim(nothing, 1e2)
ax1.set_ylim(1e-4, 1e-1)
ax1.legend(loc="lower left", ncol=2, columnspacing=-0.5, handlelength=1)

for (k, N) in enumerate([20, 50])
    new_file = "number-partitioning_" * "p_" * string(p) * "_N_" * string(N) * "_num_inst_" * string(num_instances) * "_seed_" * string(seed) * "_hist_stats.h5"
    data_file = h5open(PATH * new_file, "r")
    all_E_stars = read(data_file, "all_E_stars")
    counts, bins = np.histogram(all_E_stars, bins=100)    
    hist(N^1.9 .* bins[1:end-1], N^1.9 .* bins, weights=(counts), lw=1.5, label=@sprintf("\$N=%2d\$", N), color=colors[k], alpha=alphas[k])#, histtype="step")
    if k == 1
        plot(N^1.9 .* bins[1:end-1], map(x -> counts[1]*exp.(-x / 3.1), N^1.9 .* bins[1:end-1]), "--k")
    end
end
plot([], [], "--k", label="\$\\mathrm{e}^{- \\sqrt{E_*} N^{\\omega} / A}\$")
legend(loc="upper right", ncol=1)
ax2.set_yscale("log")
ax2.set_xlim(0, 25)
ax2.set_ylim(1e0, 1e4)
ax2.set_xlabel("\$N^{\\omega}\\sqrt{E_*}\$", labelpad=2)
ax2.set_ylabel("\$ P(\\sqrt{E_*})\$")
# savefig("../plots/" * "Fig5.pdf", dpi=300)